In [1]:
import cv2
from mtcnn import MTCNN
import random
import os
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

# Extract Frames from Videos

In [20]:

def extract_images_from_videos(videos_folder, output_folder, start_frame_number=601, num_images_per_video=10):
    frame_number = start_frame_number
    
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(videos_folder):
        if filename.endswith(".mp4"):
            video_path = os.path.join(videos_folder, filename)
            frame_number = extract_images(video_path, output_folder, start_frame_number=frame_number, num_images=num_images_per_video)

def extract_images(video_path, output_folder, num_images=30, start_frame_number=0):
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(total_frames // num_images, 1)
    count = 0
    frame_number = start_frame_number

    if not video.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return frame_number

    while True:
        ret, frame = video.read()

        if not ret:
            break

        if count % interval == 0:
            image_path = os.path.join(output_folder, f"frame_{frame_number}.jpg")
            cv2.imwrite(image_path, frame)
            print(f"Saved frame {frame_number} from {video_path}")
            frame_number += 2  # Increment by two for every next image
            if frame_number - start_frame_number >= num_images * 2:  
                break

        count += 1

    video.release()
    return frame_number

videos_folder = r"C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict"
output_folder = r"C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\frame"
start_frame_number = 0
num_images_per_video = 10

extract_images_from_videos(videos_folder, output_folder, start_frame_number, num_images_per_video)


Saved frame 0 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4
Saved frame 2 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4
Saved frame 4 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4
Saved frame 6 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4
Saved frame 8 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4
Saved frame 10 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4
Saved frame 12 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4
Saved frame 14 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4
Saved frame 16 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4
Saved frame 18 from C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\id1_id0_0000.mp4


# Extract faces and patches

In [21]:
def extract_face(image_path):
    
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found")
        return None

    # Initialize the MTCNN detector
    detector = MTCNN()

    # Detect faces in the image
    faces = detector.detect_faces(image)

    if not faces:
        print("No faces detected in the image")
        return None

     #  face with the highest confidence
    highest_confidence_face = max(faces, key=lambda x: x['confidence'])

    # Extract the bounding box coordinates
    x, y, w, h = highest_confidence_face['box']
    
    face_context = image[y:y+h, x:x+w]

    region_top = max(0, y - 139)  
    region_bottom = min(image.shape[0], y)  
    region_left = max(0, x - 139)  
    region_right = min(image.shape[1], x)  
    
    patch = image[region_top:region_bottom, region_left:region_right]
    return face_context,patch




    

In [22]:
folder_path =r"C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\frame"

faces_folder = os.path.join(folder_path, "faces")
patches_folder = os.path.join(folder_path, "patches")
os.makedirs(faces_folder, exist_ok=True)
os.makedirs(patches_folder, exist_ok=True)

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        image_path = os.path.join(folder_path, filename)
        result = extract_face(image_path)
        if result is not None:
            face, patch = result
            if face is not None and patch is not None:
                face_filename = os.path.join(faces_folder, f"face_{filename}")
                cv2.imwrite(face_filename, face)
                
                patch_filename = os.path.join(patches_folder, f"patch_{filename}")
                cv2.imwrite(patch_filename, patch)
                
                
            else:
                print(f"No face or patch found in {filename}")
        else:
            print(f"Error processing {filename}")


1/1 [==============================] - 0s 136ms/step


# Extraction of Noise features of faces and patches using inception v3

In [23]:
base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')

In [24]:
import os
from PIL import Image
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((299, 299))  # Resize to match Inception v3 input size
    image = np.array(image)
    image = preprocess_input(image)
    return image

# Function to extract features using Inception v3
def extract_features(images):
    
    features = base_model.predict(images)
    return features

# Paths to your folders
face_folders = [
    ("fake", r"C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\frame\faces"),
    
]

patch_folders = [
    ("real",r"C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict\frame\patches"),
]

# Lists to store extracted features
face_features = []
patch_features = []

# Function to process folders
def process_folders(folders, features_list):
    for category, folder_path in folders:
        for filename in os.listdir(folder_path):
            if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                image_path = os.path.join(folder_path, filename)
                image = load_and_preprocess_image(image_path)
                features = extract_features(np.expand_dims(image, axis=0))
                features_list.append((category, features))

# Process face folders
process_folders(face_folders, face_features)

# Process patch folders
process_folders(patch_folders, patch_features)




1/1 [==============================] - 0s 88ms/step


# Save features

In [25]:
def save_features(features, filename):
    
    categories, feature_arrays = zip(*features)
    # Convert categories to numpy array
    categories = np.array(categories)
    # Convert feature arrays to numpy array
    features_array = np.array(feature_arrays)
    # Save features and categories
    np.savez(filename, categories=categories, features=features_array)

    
    
# Save face features
save_features(face_features, r"C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict_faces.npz")

# Save patch features
save_features(patch_features, r"C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict_patches.npz")


In [26]:
import numpy as np

# Load face features
face_data = np.load(r"C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict_faces.npz")
face_categories = face_data['categories']
face_feature= face_data['features']

# Zip categories and features together
face_features = list(zip(face_categories, face_feature))

# Load patch features
patch_data = np.load(r"C:\Users\Sai lokesh\OneDrive\Documents\jupyter\predict_patches.npz")
patch_categories = patch_data['categories']
patch_feature = patch_data['features']
patch_features = list(zip(patch_categories, patch_feature))


# Frequency Domain Analysis.

In [27]:
def perform_fft(features):
    
  
  # Check if it's a single feature or a batch
    if len(features.shape) == 1:
        
        fft_result = fft(features)
    else:
    # Apply FFT to each feature vector in the batch
        
        fft_result = np.apply_along_axis(fft, 1, features)
    return fft_result


# Process face features
face_features_fft = perform_fft(face_feature)

# Process patch features (not used in subsequent steps based on your explanation)
patch_features_fft = perform_fft(patch_feature)

In [28]:
def extract_frequency_features(fft_result):
    

    
    magnitude_spectrum = np.abs(fft_result)

    
    frequencies = np.arange(len(magnitude_spectrum)) 
    mean_frequencies = frequencies * magnitude_spectrum / np.sum(magnitude_spectrum)
    cumulative_energy = np.cumsum(magnitude_spectrum)  # Cumulative energy across frequencies

    # Return features as arrays
    return {
        "magnitude_spectrum": magnitude_spectrum,
        "mean_frequencies": mean_frequencies,  # Array of mean frequencies
        "cumulative_energy": cumulative_energy,
    }


In [29]:
face_frequency_features = {}
for i, feature_fft in enumerate(face_features_fft):
  # Extract features for each face feature
  face_frequency_features[i] = extract_frequency_features(feature_fft)

In [30]:
patch_frequency_features = {}
for i, feature_fft in enumerate(patch_features_fft):
  # Extract features for each patch feature
  patch_frequency_features[i] = extract_frequency_features(feature_fft)

# Concatenate Noise and frequency Features

In [31]:
# Concatenate features for each sample
combined_features = []
for i in range(len(face_feature)):
    
    
    combined_feature = np.concatenate((face_feature[i], patch_feature[i], face_frequency_features[i]["magnitude_spectrum"], patch_frequency_features[i]["magnitude_spectrum"]))
    
    combined_features.append(combined_feature)

# Convert to numpy array for better handling
combined_features = np.array(combined_features)

print("Combined features shape:", combined_features.shape)

Combined features shape: (10, 4, 2048)


# Load the Saved Model

In [32]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model("siam_net_model4.h5")

In [33]:
x = combined_features.reshape(combined_features.shape[0], -1)

# Prediction

In [34]:
predictions = loaded_model.predict(x)

1/1 [==============================] - 0s 72ms/step


In [35]:
binary_predictions = (predictions > 0.5).astype(int)

In [36]:
binary_predictions

array([[1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1]])

# Output
1 indicates the video is deepfake.

0 indicates the video is real

In [37]:
unique, counts = np.unique(binary_predictions, return_counts=True)
counts_dict = dict(zip(unique, counts))

# Determine majority value
majority_value = max(counts_dict, key=counts_dict.get)

print("predicted:", majority_value)

predicted: 1
